# GitBook MD 파일 스마트 분할기

GitBook의 특성을 고려한 효과적인 마크다운 분할 전략

In [ ]:
import os
import re
from typing import List, Dict, Any, Tuple
from pathlib import Path
from langchain.docstore.document import Document
from langchain_text_splitters import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter,
    Language
)
import json

In [ ]:
class GitBookSmartSplitter:
    """GitBook 특화 스마트 분할기"""
    
    def __init__(self):
        self.header_splitter = MarkdownHeaderTextSplitter(
            headers_to_split_on=[
                ("#", "title"),
                ("##", "section"),
                ("###", "subsection"),
                ("####", "subsubsection"),
                ("#####", "subsubsubsection")
            ]
        )
        
        self.recursive_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            separators=["\n\n", "\n", ". ", " ", ""],
            length_function=len,
            is_separator_regex=False
        )
    
    def preprocess_gitbook_content(self, content: str) -> str:
        """GitBook 콘텐츠 전처리"""
        # GitBook 특수 태그 제거
        content = re.sub(r'{%[^%]*%}', '', content)  # Liquid 태그
        content = re.sub(r'\{\{[^}]*\}\}', '', content)  # Handlebars
        content = re.sub(r'<[^>]*>', '', content)  # HTML 태그
        
        # GitBook 링크 정리
        content = re.sub(r'\[([^\]]+)\]\([^)]*\)', r'\1', content)  # 링크 텍스트만 유지
        
        # 불필요한 공백 정리
        content = re.sub(r'\n\s*\n\s*\n', '\n\n', content)
        
        return content.strip()
    
    def extract_gitbook_metadata(self, content: str) -> Dict[str, Any]:
        """GitBook 메타데이터 추출"""
        metadata = {
            'has_code_blocks': False,
            'has_tables': False,
            'has_images': False,
            'has_links': False,
            'header_count': 0,
            'code_languages': [],
            'estimated_complexity': 'low'
        }
        
        # 코드 블록 확인
        code_blocks = re.findall(r'```(\w+)?\n([\s\S]*?)```', content)
        if code_blocks:
            metadata['has_code_blocks'] = True
            metadata['code_languages'] = [lang for lang, _ in code_blocks if lang]
        
        # 테이블 확인
        if '|' in content and '---' in content:
            metadata['has_tables'] = True
        
        # 이미지 확인
        if '![' in content:
            metadata['has_images'] = True
        
        # 링크 확인
        if '[' in content and '](' in content:
            metadata['has_links'] = True
        
        # 헤더 개수
        metadata['header_count'] = len(re.findall(r'^#{1,6}\s+', content, re.MULTILINE))
        
        # 복잡도 추정
        if metadata['has_code_blocks'] and metadata['has_tables']:
            metadata['estimated_complexity'] = 'high'
        elif metadata['has_code_blocks'] or metadata['has_tables']:
            metadata['estimated_complexity'] = 'medium'
        
        return metadata
    
    def split_by_gitbook_structure(self, content: str, source: str = "") -> List[Document]:
        """GitBook 구조에 따른 분할"""
        # 1단계: 헤더 기반 분할
        header_docs = self.header_splitter.split_text(content)
        
        documents = []
        for i, doc in enumerate(header_docs):
            # 메타데이터 추출
            metadata = self.extract_gitbook_metadata(doc.page_content)
            
            # GitBook 특성에 따른 추가 메타데이터
            doc_metadata = {
                **doc.metadata,
                **metadata,
                'source': source,
                'split_method': 'header_based',
                'chunk_idx': i,
                'gitbook_processed': True
            }
            
            # 새로운 Document 생성
            new_doc = Document(
                page_content=doc.page_content,
                metadata=doc_metadata
            )
            documents.append(new_doc)
        
        return documents
    
    def split_by_content_type(self, content: str, source: str = "") -> List[Document]:
        """콘텐츠 타입에 따른 분할"""
        documents = []
        
        # 코드 블록 분리
        code_pattern = r'```(\w+)?\n([\s\S]*?)```'
        code_blocks = re.findall(code_pattern, content)
        
        # 코드 블록을 임시로 마스킹
        masked_content = re.sub(code_pattern, '___CODE_BLOCK___', content)
        
        # 일반 텍스트 분할
        text_chunks = self.recursive_splitter.split_text(masked_content)
        
        # 코드 블록 복원 및 Document 생성
        code_idx = 0
        for i, chunk in enumerate(text_chunks):
            # 코드 블록 복원
            restored_chunk = chunk
            while '___CODE_BLOCK___' in restored_chunk and code_idx < len(code_blocks):
                lang, code = code_blocks[code_idx]
                code_block = f"```{lang}\n{code}\n```"
                restored_chunk = restored_chunk.replace('___CODE_BLOCK___', code_block, 1)
                code_idx += 1
            
            # 메타데이터 생성
            metadata = self.extract_gitbook_metadata(restored_chunk)
            
            doc = Document(
                page_content=restored_chunk,
                metadata={
                    **metadata,
                    'source': source,
                    'split_method': 'content_type_based',
                    'chunk_idx': i,
                    'gitbook_processed': True
                }
            )
            documents.append(doc)
        
        return documents
    
    def split_by_semantic_units(self, content: str, source: str = "") -> List[Document]:
        """의미적 단위로 분할"""
        documents = []
        
        # 섹션별로 분할
        sections = re.split(r'(?=^#{1,6}\s+)', content, flags=re.MULTILINE)
        
        for i, section in enumerate(sections):
            if not section.strip():
                continue
            
            # 섹션 내에서 더 세분화
            if len(section) > 2000:  # 큰 섹션은 다시 분할
                subsections = self.recursive_splitter.split_text(section)
                for j, subsection in enumerate(subsections):
                    metadata = self.extract_gitbook_metadata(subsection)
                    
                    doc = Document(
                        page_content=subsection,
                        metadata={
                            **metadata,
                            'source': source,
                            'split_method': 'semantic_units',
                            'section_idx': i,
                            'subsection_idx': j,
                            'gitbook_processed': True
                        }
                    )
                    documents.append(doc)
            else:
                metadata = self.extract_gitbook_metadata(section)
                
                doc = Document(
                    page_content=section,
                    metadata={
                        **metadata,
                        'source': source,
                        'split_method': 'semantic_units',
                        'section_idx': i,
                        'gitbook_processed': True
                    }
                )
                documents.append(doc)
        
        return documents
    
    def smart_split(self, content: str, source: str = "", strategy: str = "auto") -> List[Document]:
        """스마트 분할 메인 함수"""
        # 전처리
        processed_content = self.preprocess_gitbook_content(content)
        
        # 메타데이터 분석
        metadata = self.extract_gitbook_metadata(processed_content)
        
        # 전략 선택
        if strategy == "auto":
            if metadata['header_count'] > 5:
                strategy = "structure"
            elif metadata['has_code_blocks'] and metadata['estimated_complexity'] == 'high':
                strategy = "content_type"
            else:
                strategy = "semantic"
        
        # 분할 실행
        if strategy == "structure":
            documents = self.split_by_gitbook_structure(processed_content, source)
        elif strategy == "content_type":
            documents = self.split_by_content_type(processed_content, source)
        elif strategy == "semantic":
            documents = self.split_by_semantic_units(processed_content, source)
        else:
            raise ValueError(f"Unknown strategy: {strategy}")
        
        # 결과 통계
        print(f"📊 분할 결과:")
        print(f"  - 전략: {strategy}")
        print(f"  - 문서 수: {len(documents)}")
        print(f"  - 평균 길이: {sum(len(doc.page_content) for doc in documents) // len(documents)} 문자")
        print(f"  - 복잡도: {metadata['estimated_complexity']}")
        
        return documents

In [ ]:
# GitBook 스마트 분할기 테스트
def load_test_gitbook_content() -> str:
    """테스트용 GitBook 콘텐츠 로드"""
    # 실제 GitBook 콘텐츠 예시
    test_content = """
# 원스토어 인앱결제 API V7(SDK V21)

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.

## 개요

이 문서는 원스토어 인앱결제 API V7의 사용법을 설명합니다.

### 주요 기능

- API V7 지원
- SDK V21 통합
- 향상된 보안

## 설치 방법

### Gradle 설정

```gradle
dependencies {
    implementation 'com.onestore:iap:21.0.0'
}
```

### 초기화 코드

```kotlin
val purchaseClient = PurchaseClient.newBuilder()
    .setListener(purchasesUpdatedListener)
    .build()
```

## 구매 프로세스

### 1. 상품 조회

```kotlin
purchaseClient.queryProductDetailsAsync(
    QueryProductDetailsParams.newBuilder()
        .setProductIds(listOf("product_id"))
        .build()
)
```

### 2. 구매 실행

```kotlin
purchaseClient.launchPurchaseFlow(
    activity,
    PurchaseFlowParams.newBuilder()
        .setProductId("product_id")
        .build()
)
```

## 테이블 예시

| 기능 | 설명 | 버전 |
|------|------|------|
| API V7 | 최신 API | 21.0.0 |
| SDK V21 | 통합 SDK | 21.0.0 |

## 이미지 예시

![결제 화면](payment_screen.png)

## 링크 예시

[공식 문서](https://onestore-dev.gitbook.io)

{% hint style="info" %}
이것은 GitBook 힌트입니다.
{% endhint %}

{% hint style="warning" %}
주의사항입니다.
{% endhint %}
"""
    return test_content

# 스마트 분할기 초기화
splitter = GitBookSmartSplitter()

# 테스트 콘텐츠 로드
test_content = load_test_gitbook_content()
print(f"📄 테스트 콘텐츠 크기: {len(test_content)} 문자")
print(f"📄 테스트 콘텐츠 미리보기:")
print("=" * 60)
print(test_content[:500] + "...")
print("=" * 60)

In [ ]:
# 다양한 전략으로 분할 테스트
strategies = ["auto", "structure", "content_type", "semantic"]

for strategy in strategies:
    print(f"\n🔧 전략: {strategy}")
    print("-" * 40)
    
    documents = splitter.smart_split(test_content, source="test_gitbook.md", strategy=strategy)
    
    # 결과 분석
    for i, doc in enumerate(documents[:3], 1):  # 처음 3개만 출력
        print(f"\n📄 문서 {i}:")
        print(f"  길이: {len(doc.page_content)} 문자")
        print(f"  분할 방법: {doc.metadata.get('split_method', 'unknown')}")
        print(f"  코드 블록: {doc.metadata.get('has_code_blocks', False)}")
        print(f"  복잡도: {doc.metadata.get('estimated_complexity', 'unknown')}")
        print(f"  내용: {doc.page_content[:100]}...")
    
    if len(documents) > 3:
        print(f"\n... 및 {len(documents) - 3}개 더")

In [ ]:
# 실제 GitBook 파일 처리
def process_gitbook_file(file_path: str, output_dir: str = "data/gitbook_splits"):
    """실제 GitBook 파일을 처리합니다."""
    if not os.path.exists(file_path):
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return
    
    # 파일 로드
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(f"📄 파일 로드 완료: {len(content)} 문자")
    
    # 스마트 분할
    documents = splitter.smart_split(content, source=file_path, strategy="auto")
    
    # 결과 저장
    os.makedirs(output_dir, exist_ok=True)
    
    # JSON 형태로 저장
    docs_data = []
    for i, doc in enumerate(documents):
        doc_data = {
            'id': i,
            'content': doc.page_content,
            'metadata': doc.metadata
        }
        docs_data.append(doc_data)
    
    output_file = os.path.join(output_dir, f"{Path(file_path).stem}_split.json")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(docs_data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 분할 결과 저장 완료: {output_file}")
    
    # 통계 출력
    print(f"\n📊 분할 통계:")
    print(f"  - 총 문서 수: {len(documents)}")
    print(f"  - 평균 길이: {sum(len(doc.page_content) for doc in documents) // len(documents)} 문자")
    print(f"  - 최소 길이: {min(len(doc.page_content) for doc in documents)} 문자")
    print(f"  - 최대 길이: {max(len(doc.page_content) for doc in documents)} 문자")
    
    # 메타데이터 통계
    code_blocks_count = sum(1 for doc in documents if doc.metadata.get('has_code_blocks', False))
    tables_count = sum(1 for doc in documents if doc.metadata.get('has_tables', False))
    
    print(f"  - 코드 블록 포함: {code_blocks_count}개")
    print(f"  - 테이블 포함: {tables_count}개")
    
    return documents

# 실제 파일 처리 (예시)
test_files = [
    "data/iap_gitbook.md",  # 이전에 생성한 통합 파일
    "data/dev_center_guide_touched.md"  # 기존 파일
]

for file_path in test_files:
    if os.path.exists(file_path):
        print(f"\n🔧 처리 중: {file_path}")
        print("=" * 60)
        documents = process_gitbook_file(file_path)
    else:
        print(f"\n⚠️ 파일이 없습니다: {file_path}")

In [ ]:
# 분할 품질 평가
def evaluate_split_quality(documents: List[Document]) -> Dict[str, Any]:
    """분할 품질을 평가합니다."""
    
    # 길이 분포 분석
    lengths = [len(doc.page_content) for doc in documents]
    avg_length = sum(lengths) / len(lengths)
    
    # 키워드 보존률 확인
    important_keywords = [
        "launchPurchaseFlow", "PurchaseClient", "IapResult",
        "API", "SDK", "결제", "구매"
    ]
    
    keyword_preservation = {}
    for keyword in important_keywords:
        docs_with_keyword = [
            doc for doc in documents 
            if keyword.lower() in doc.page_content.lower()
        ]
        keyword_preservation[keyword] = len(docs_with_keyword)
    
    # 메타데이터 품질
    metadata_completeness = sum(
        1 for doc in documents 
        if doc.metadata.get('gitbook_processed', False)
    ) / len(documents)
    
    # 중복 콘텐츠 확인
    content_hashes = set()
    duplicates = 0
    for doc in documents:
        content_hash = hash(doc.page_content[:100])  # 처음 100자로 해시
        if content_hash in content_hashes:
            duplicates += 1
        content_hashes.add(content_hash)
    
    duplicate_rate = duplicates / len(documents) if documents else 0
    
    return {
        'total_documents': len(documents),
        'average_length': avg_length,
        'length_variance': sum((l - avg_length) ** 2 for l in lengths) / len(lengths),
        'keyword_preservation': keyword_preservation,
        'metadata_completeness': metadata_completeness,
        'duplicate_rate': duplicate_rate,
        'quality_score': (1 - duplicate_rate) * metadata_completeness * 100
    }

# 품질 평가 실행
if os.path.exists("data/iap_gitbook.md"):
    with open("data/iap_gitbook.md", 'r', encoding='utf-8') as f:
        content = f.read()
    
    documents = splitter.smart_split(content, source="iap_gitbook.md", strategy="auto")
    
    quality_metrics = evaluate_split_quality(documents)
    
    print(f"📊 분할 품질 평가 결과:")
    print(f"  - 총 문서 수: {quality_metrics['total_documents']}")
    print(f"  - 평균 길이: {quality_metrics['average_length']:.1f} 문자")
    print(f"  - 길이 분산: {quality_metrics['length_variance']:.1f}")
    print(f"  - 메타데이터 완성도: {quality_metrics['metadata_completeness']:.2f}")
    print(f"  - 중복률: {quality_metrics['duplicate_rate']:.2f}")
    print(f"  - 품질 점수: {quality_metrics['quality_score']:.1f}/100")
    
    print(f"\n🔍 키워드 보존률:")
    for keyword, count in quality_metrics['keyword_preservation'].items():
        print(f"  - {keyword}: {count}개 문서에서 발견")
else:
    print("❌ 평가할 파일이 없습니다.")

In [ ]:
# RAG 최적화 분할
def create_rag_optimized_splits(content: str, source: str = "") -> List[Document]:
    """RAG 시스템에 최적화된 분할"""
    
    # 1단계: 기본 스마트 분할
    base_docs = splitter.smart_split(content, source, strategy="auto")
    
    # 2단계: RAG 최적화
    optimized_docs = []
    
    for doc in base_docs:
        # 너무 작은 청크는 건너뛰기
        if len(doc.page_content) < 100:
            continue
        
        # 너무 큰 청크는 재분할
        if len(doc.page_content) > 2000:
            sub_docs = splitter.recursive_splitter.split_text(doc.page_content)
            for i, sub_content in enumerate(sub_docs):
                if len(sub_content) >= 100:  # 최소 길이 확인
                    sub_metadata = {
                        **doc.metadata,
                        'split_method': 'rag_optimized',
                        'sub_chunk_idx': i,
                        'original_length': len(doc.page_content)
                    }
                    
                    optimized_docs.append(Document(
                        page_content=sub_content,
                        metadata=sub_metadata
                    ))
        else:
            # 적절한 크기는 그대로 사용
            doc.metadata['split_method'] = 'rag_optimized'
            optimized_docs.append(doc)
    
    # 3단계: 키워드 강화
    for doc in optimized_docs:
        # 중요 키워드가 포함된 문서는 우선순위 부여
        important_keywords = [
            "launchPurchaseFlow", "PurchaseClient", "IapResult",
            "API", "SDK", "결제", "구매"
        ]
        
        keyword_count = sum(
            1 for keyword in important_keywords
            if keyword.lower() in doc.page_content.lower()
        )
        
        doc.metadata['keyword_score'] = keyword_count
        doc.metadata['is_important'] = keyword_count >= 2
    
    print(f"✅ RAG 최적화 완료: {len(optimized_docs)}개 문서")
    print(f"  - 중요 문서: {sum(1 for doc in optimized_docs if doc.metadata.get('is_important', False))}개")
    
    return optimized_docs

# RAG 최적화 분할 테스트
if os.path.exists("data/iap_gitbook.md"):
    with open("data/iap_gitbook.md", 'r', encoding='utf-8') as f:
        content = f.read()
    
    rag_docs = create_rag_optimized_splits(content, "iap_gitbook.md")
    
    # 결과 저장
    output_file = "data/rag_optimized_splits.json"
    docs_data = [
        {
            'id': i,
            'content': doc.page_content,
            'metadata': doc.metadata
        }
        for i, doc in enumerate(rag_docs)
    ]
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(docs_data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ RAG 최적화 결과 저장: {output_file}")
else:
    print("❌ RAG 최적화를 위한 파일이 없습니다.")